In [210]:
import pandas as pd
import numpy as np
import pyarrow as pa
import re
import findspark
findspark.init('/home/dave/spark-2.4.1-bin-hadoop2.7/')
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import *
import sys
import os
from IPython.display import display, HTML
from pyspark.sql.functions import pandas_udf, PandasUDFType

spark = SparkSession.builder.appName("proj_exp").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

datapath = "/home/dave/Documents/CS691/proj/data/snotel/"
s1049="SNOTEL_1049_FORESTDALE_CREEK"    
s539="SNOTEL_539_INDEPENDENCE_CAMP"
s1050="SNOTEL_1050_HORSE_MEADOW"        
s540="SNOTEL_540_INDEPENDENCE_CREEK"
s1051="SNOTEL_1051_BURNSIDE_LAKE"       
s541="SNOTEL_541_INDEPENDENCE_LAKE"
s1067="SNOTEL_1067_CARSON_PASS"         
s615="SNOTEL_615_MARLETTE_LAKE"
s340="SNOTEL_340_BIG_MEADOW"           
s652="SNOTEL_652_MT_ROSE"
s356="SNOTEL_356_BLUE_LAKES"          
s428="SNOTEL_428_CSS_LAB"              
s724="SNOTEL_724_RUBICON"
s809="SNOTEL_809_TAHOE_CITY"
s463="SNOTEL_463_ECHO_PEAK"            
s834="SNOTEL_834_TRUCKEE"
s508="SNOTEL_508_HAGANS_MEADOW"        
s848="SNOTEL_848_WARD_CREEK"
s518="SNOTEL_518_HEAVENLY_VALLEY"

mt_rose = [s340,s652,s615]
hope_valley = [s1051,s1049,s1067,s356]

truckee = [s541,s540,s428]
df541 = pd.read_csv(s541+".csv")
df540 = pd.read_csv(s540+".csv")
df428 = pd.read_csv(s428+".csv")
df541 = df541.merge(df540, on = "date_only")
truckee = df541.merge(df428, on = "date_only")
truckee =truckee.loc[:, ~truckee.columns.str.contains('^Unnamed')]
truckee['mean'] = truckee.mean(axis =1)
truckee['mean'] = truckee['mean'].round()
truckee = truckee.drop(['max(total_av_corrected)_x','max(total_av_corrected)_y','max(total_av_corrected)'], axis =1)

west_shore = [s809,s724]
df809 = pd.read_csv(s809+".csv")
df724 = pd.read_csv(s724+".csv")
west = df809.rename(index=str,columns={"max(total_av_corrected)":"mean"})

south_shore = [s518,s508,s1050,s463]
df518 = pd.read_csv(s518+".csv")
df508 = pd.read_csv(s508+".csv")
df1050 = pd.read_csv(s1050+".csv")
df463 = pd.read_csv(s463+".csv")
df518 = df518.merge(df463,on = "date_only")
df518 = df518.merge(df1050,on = "date_only")
south = df518.merge(df508, on = "date_only")
south =south.loc[:, ~south.columns.str.contains('^Unnamed')]
south['mean'] = south.mean(axis =1)
south['mean'] = south['mean'].round()
south = south.drop(['max(total_av_corrected)_x','max(total_av_corrected)_y'], axis =1)

hope_valley = [s1051,s1049,s1067,s356]
df1051 = pd.read_csv(s1051+".csv")
df1049 = pd.read_csv(s1049+".csv")
df1067 = pd.read_csv(s1067+".csv")
df356 = pd.read_csv(s356+".csv")
df1049 = df1049.merge(df1051, on = "date_only")
df1067 = df1067.merge(df1049, on = "date_only")
hope = df1067.merge(df356, on = "date_only")
hope =hope.loc[:, ~hope.columns.str.contains('^Unnamed')]
hope['mean'] = hope.mean(axis =1)
hope = hope.drop(['max(total_av_corrected)_x','max(total_av_corrected)_y'], axis =1)


mt_rose = [s340,s652,s615]
df340 = pd.read_csv(s340+".csv")
#df652 = pd.read_csv(s652+".csv")
df615 = pd.read_csv(s615+".csv")
#df340 = df340.merge(df652, on = "date_only")
rose = df340.merge(df615, on = "date_only")
rose =rose.loc[:, ~rose.columns.str.contains('^Unnamed')]
rose['mean'] = rose.mean(axis =1)
rose['mean'] = rose['mean'].round()
rose = rose.drop(['max(total_av_corrected)_x','max(total_av_corrected)_y'], axis =1)

agg = rose.merge(hope, on = "date_only")
agg = agg.merge(south, on = "date_only")
agg = agg.merge(west, on = "date_only")
agg = agg.merge(truckee, on = "date_only")

agg =agg.loc[:, ~agg.columns.str.contains('^Unnamed')]
agg['forecast'] = agg[['mean_x','mean_y','mean_x','mean_y','mean']].max(axis=1)
agg = agg.drop(['mean_x','mean_y','mean_x','mean_y','mean'], axis =1)

sac = pd.read_csv("sac.csv")
sac = sac.drop(['Unnamed: 0','1','2','3'], axis =1)

#sac = sac.replace('January', '1')
#sac = sac.replace('February', '2')
#sac = sac.replace('March', '3')
#sac = sac.replace(to_replace=r'April', value='4', regex=True)
#sac = sac.replace(to_replace=r'April', value='4', regex=True)
#sac = sac.replace(to_replace=r'March', value='3', regex=True)
#sac = sac.replace(to_replace=r'April', value='4', regex=True)
#sac = sac.replace('May', '5')
df = spark.createDataFrame(sac)
df2 = spark.createDataFrame(agg)
#df = df.withColumn('timestamp',F.col('0').cast("timestamp"))

split_col = pyspark.sql.functions.split(df['0'], 'at')
df = df.withColumn('Date', split_col.getItem(0))
df = df.select('Date', 'av')
#df = df.withColumn('Time', split_col.getItem(1))
#df = df.select('date_str', from_unixtime(unix_timestamp('date_str', 'MM dd, yyyy')).alias('date'))
#df = df.withColumn('timestamp',F.col('Date').cast("timestamp"))

df = df.filter(df.Date.rlike('.*Feb.*2019'))
df2 = df2.filter(df2.date_only.rlike('.*2019-02.*')).orderBy('date_only', ascending = False)

df2 = df2.withColumn("id", F.row_number().over(Window.orderBy("date_only")))
df = df.withColumn("id",F.monotonically_increasing_id())
#df2 = df2.withColumn("id",F.monotonically_increasing_id())

df = df.join(df2,['id']).orderBy('id')
df.select('id','av',fore)
df.show() 




+---+------------------+---+----------+--------+
| id|              Date| av| date_only|forecast|
+---+------------------+---+----------+--------+
| 26| February 4, 2019 |  4|2019-02-26|     4.0|
| 19|February 10, 2019 |  3|2019-02-19|     2.0|
| 22| February 8, 2019 |  2|2019-02-22|    2.25|
|  7|February 22, 2019 |  2|2019-02-07|     1.5|
| 25| February 5, 2019 |  3|2019-02-25|     3.0|
|  6|February 23, 2019 |  1|2019-02-06|     4.0|
|  9|February 20, 2019 |  2|2019-02-09|     3.0|
| 27| February 3, 2019 |  4|2019-02-27|     4.0|
| 17|February 12, 2019 |  2|2019-02-17|     5.0|
| 28| February 2, 2019 |  4|2019-02-28|     3.0|
|  5|February 24, 2019 |  2|2019-02-05|     5.0|
|  1|February 27, 2019 |  4|2019-02-01|     3.0|
| 10|February 19, 2019 |  2|2019-02-10|     5.0|
|  3|February 25, 2019 |  4|2019-02-03|    4.25|
| 12|February 17, 2019 |  3|2019-02-12|    3.25|
|  8|February 21, 2019 |  2|2019-02-08|     1.0|
| 11|February 18, 2019 |  2|2019-02-11|     4.0|
|  2|February 26, 20

In [146]:
agg

,date_only,forecast
0,2009-04-01,2.00
1,2009-04-02,3.00
2,2009-04-03,3.00
3,2009-04-04,4.00
4,2009-04-05,4.00
5,2009-04-06,4.00
6,2009-04-07,4.00
7,2009-04-08,4.50
8,2009-04-09,3.75
9,2009-04-10,5.00


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 2 columns):
date_only    3653 non-null object
mean         3653 non-null float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,date_only,mean
0,2009-04-01,2.0
1,2009-04-02,2.0
2,2009-04-03,2.0
3,2009-04-04,2.0
4,2009-04-05,3.0
5,2009-04-06,4.0
6,2009-04-07,4.0
7,2009-04-08,2.0
8,2009-04-09,2.0
9,2009-04-10,2.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 2 columns):
date_only    3653 non-null object
mean         3653 non-null float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,date_only,mean
0,2009-04-01,2.0
1,2009-04-02,3.0
2,2009-04-03,3.0
3,2009-04-04,4.0
4,2009-04-05,4.0
5,2009-04-06,4.0
6,2009-04-07,4.0
7,2009-04-08,2.0
8,2009-04-09,2.0
9,2009-04-10,2.0


,Unnamed: 0,date_only,mean
0,0,2009-04-01,1.0
1,1,2009-04-02,2.0
2,2,2009-04-03,2.0
3,3,2009-04-04,4.0
4,4,2009-04-05,4.0
5,5,2009-04-06,4.0
6,6,2009-04-07,3.0
7,7,2009-04-08,3.0
8,8,2009-04-09,3.0
9,9,2009-04-10,5.0


/home/dave/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/dave/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


KeyError: "labels ['max(total_av_corrected)_x' 'max(total_av_corrected)_y'] not contained in axis"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 2 columns):
date_only    3653 non-null object
mean         3653 non-null float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,date_only,mean
0,2009-04-01,2.0
1,2009-04-02,2.0
2,2009-04-03,2.0
3,2009-04-04,3.0
4,2009-04-05,4.0
5,2009-04-06,4.0
6,2009-04-07,4.0
7,2009-04-08,2.0
8,2009-04-09,2.0
9,2009-04-10,1.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 2 columns):
date_only    3653 non-null object
mean         3653 non-null float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,date_only,mean
0,2009-04-01,1.75
1,2009-04-02,2.00
2,2009-04-03,2.25
3,2009-04-04,2.75
4,2009-04-05,2.75
5,2009-04-06,3.00
6,2009-04-07,3.00
7,2009-04-08,4.50
8,2009-04-09,3.75
9,2009-04-10,3.25


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3653 entries, 0 to 3652
Data columns (total 2 columns):
date_only    3653 non-null object
mean         3653 non-null float64
dtypes: float64(1), object(1)
memory usage: 85.6+ KB


,date_only,mean
0,2009-04-01,2.0
1,2009-04-02,2.0
2,2009-04-03,2.0
3,2009-04-04,3.0
4,2009-04-05,4.0
5,2009-04-06,4.0
6,2009-04-07,4.0
7,2009-04-08,2.0
8,2009-04-09,2.0
9,2009-04-10,1.0


ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

,Unnamed: 0_x,date_only,max(total_av_corrected)_x,Unnamed: 0_y,max(total_av_corrected)_y,Unnamed: 0,max(total_av_corrected),mean
0,0,2009-04-01,2.0,0,2.0,0,3.0,1.166667
1,1,2009-04-02,1.0,1,3.0,1,5.0,2.000000
2,2,2009-04-03,2.0,2,1.0,2,5.0,2.333333
3,3,2009-04-04,2.0,3,5.0,3,5.0,3.500000
4,4,2009-04-05,2.0,4,5.0,4,4.0,3.833333
5,5,2009-04-06,2.0,5,5.0,5,5.0,4.500000
6,6,2009-04-07,2.0,6,5.0,6,5.0,5.000000
7,7,2009-04-08,1.0,7,2.0,7,4.0,4.666667
8,8,2009-04-09,1.0,8,1.0,8,5.0,5.166667
9,9,2009-04-10,1.0,9,1.0,9,5.0,5.666667


,Unnamed: 0,date_only,max(total_av_corrected)
0,0,2009-04-01,2.0
1,1,2009-04-02,1.0
2,2,2009-04-03,2.0
3,3,2009-04-04,2.0
4,4,2009-04-05,2.0
5,5,2009-04-06,2.0
6,6,2009-04-07,2.0
7,7,2009-04-08,1.0
8,8,2009-04-09,1.0
9,9,2009-04-10,1.0


,Unnamed: 0,date_only,max(total_av_corrected)
0,0,2009-04-01,2.0
1,1,2009-04-02,3.0
2,2,2009-04-03,1.0
3,3,2009-04-04,5.0
4,4,2009-04-05,5.0
5,5,2009-04-06,5.0
6,6,2009-04-07,5.0
7,7,2009-04-08,2.0
8,8,2009-04-09,1.0
9,9,2009-04-10,1.0


,Unnamed: 0,date_only,max(total_av_corrected)
0,0,2009-04-01,3.0
1,1,2009-04-02,5.0
2,2,2009-04-03,5.0
3,3,2009-04-04,5.0
4,4,2009-04-05,4.0
5,5,2009-04-06,5.0
6,6,2009-04-07,5.0
7,7,2009-04-08,4.0
8,8,2009-04-09,5.0
9,9,2009-04-10,5.0


,Unnamed: 0,date_only,max(total_av_corrected)
